# Data to LED Examples

This notebook will adjust LEDS connected to pins 18-21

# Steps to use a numbered GPIO PIN
1. import GPIO library
2. clear the warnings
3. set GPIO.BCM mode
4. set the pin as output


In [10]:
print ("Current working directory:")
!pwd
print ("\nAvailable files:")
!ls *.csv

Current working directory:
/home/pi/codeattacc-connected/Notebook

Available files:
Full-Turkey-Creek-Sensor-Data.csv
recordings-073021-114357.csv
recordings-073021-151701.csv
SAMPLE_recording_8-072422-223527.csv
SAMPLE-recordings-073021-114357.csv
SAMPLE-rpiz_sensor-1-20180719-sensorlog.csv
site2.csv


In [12]:
# Insert the file name
datafile = 'recordings-073021-151701.csv'


In [13]:
# Libraries needed to do the work
import pandas as pd # The Pandas library makes it easier to work with data
import numpy as np # Numpy adds additional numerical methods (math functions)
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')

# Read in the csv datafile from above, and save it to the Pandas "dataframe" named sensor_data
# and add the header names to match the data
sensor_data = pd.read_csv(datafile)

# Print the first 5 lines of the dataframe named sensor_data
sensor_data.head()

,recording,date,time,temperature,humidity,pressure,lux,oxidizing,reducing,nh3,noise_amp,pm1_0,pm2_5,pm_10
0,recording_3,07/25/22,16:49:54,72.496087,46.611444,1000.068354,65.06325,9776.828617,767.653496,728.444803,24.021394,4.0,4.0,4.0
1,recording_3,07/25/22,16:49:55,92.807560,45.907393,1000.074148,69.15195,659.959759,165532.928748,1623.947615,16.958580,5.0,5.0,5.0
2,recording_3,07/25/22,16:49:57,92.767447,45.978827,1000.084502,62.27830,682.769726,257685.550605,3502.535619,19.356770,5.0,6.0,6.0
3,recording_3,07/25/22,16:49:59,92.702695,45.823401,1000.088602,58.60430,728.444803,328549.356223,5523.096130,20.851212,5.0,6.0,6.0
4,recording_3,07/25/22,16:50:01,92.632213,45.089221,1000.092257,65.30020,780.735108,395902.796882,7983.380940,25.476972,5.0,6.0,6.0


In [14]:
import RPi.GPIO as GPIO   ## Library that allows the GPIO pins to be used
import time
##Setup the GPIO Pins
for pin in range(18,22):
    GPIO.setwarnings(False)  ## Removes any warnings if the pin was previously used
    GPIO.setmode(GPIO.BCM)   ## Allows the use of the GPIO pin numbers instead of the physical pin numbers
    GPIO.setup(pin,GPIO.OUT)
    GPIO.output(pin,GPIO.LOW) ## Sets GPIO pin 21 output voltage low/off

# Lights adjust based on the Humidity readings

More leds will turn on based on the humidity reading found in the data set

In [15]:
max_humidity = sensor_data["humidity"].max()
print("The maximum humidity in the data is: ", max_humidity)

The maximum humidity in the data is:  48.66200206928198


In [16]:
# Using NumPY Interpolation the below interpolates the sensors data into a given range which in
# this case are the number of LEDS connected
# np.interp(sensorvalue, [min of all sensorvalues, max of all sensorvalues] , [first led pin, last led pin])
int(np.interp(max_humidity, [sensor_data["humidity"].min(),sensor_data["humidity"].max()], [18,21]))

21

In [23]:
## Cycle through all the values in the data set under Humidity
for sensorvalue in sensor_data["humidity"]:
    #Interpolate the current value
    pin = int(np.interp(sensorvalue, [sensor_data["humidity"].min(),sensor_data["humidity"].max()], [18,21]))
    #Turn on the number of pins up to the interpolated value
    for pins in range(18, pin +1):
        GPIO.output(pins,GPIO.HIGH)
    time.sleep(2)
    #Turn all the pins off before checking the next value
    for pin_off in range(18,22):
        GPIO.output(pin_off,GPIO.LOW)
    time.sleep(1)
# Turn all pins off
for pin_off in range(18,22):
        GPIO.output(pin_off,GPIO.LOW)

KeyboardInterrupt: 

# Stop Light Colors from Data

Connect:
* Red LED to pin 18
* Yellow LED to pin 19
* Green LED to pin 20

In [18]:
#Using Analog channel 0 (A0) for this example
# np.interp(sensorvalue, [min of all sensorvalues, max of all sensorvalues] , [first led pin, last led pin])
max_a0 = sensor_data["lux"].max()
print("The maximum value for lux in the data is: ", max_a0)
print(int(np.interp(max_a0, [sensor_data["lux"].min(),sensor_data["lux"].max()], [18,20])))
min_a0 = sensor_data["lux"].min()
print("The min value for lux in the data is: ", min_a0)
print(int(np.interp(min_a0, [sensor_data["lux"].min(),sensor_data["lux"].max()], [18,20])))
mean_a0 = sensor_data["lux"].mean()
print("The mean value for lux in the data is: ", mean_a0)
print(int(np.interp(mean_a0, [sensor_data["lux"].min(),sensor_data["lux"].max()], [18,20])))

The maximum value for lux in the data is:  117.09125
20
The min value for lux in the data is:  35.4939
18
The mean value for lux in the data is:  77.31777499999998
19


#### Note: Analog sensors generally have higher readings when nothing is detected and lower reading when something is detected. 

In [21]:
# An easily changeable value to pick which sensor to use
sensor_value_name = "nh3"
pins_to_use = [18,20]
# Cycle through all the values in the data set under Humidity
for sensorvalue in sensor_data[sensor_value_name]:
    #Interpolate the current value
    pin = int(np.interp(sensorvalue, [sensor_data[sensor_value_name].min(),sensor_data[sensor_value_name].max()], pins_to_use))
    for pin_num in range(18,pin +1):
        GPIO.output(pin_num,GPIO.HIGH)
    time.sleep(2)
    #Turn all the pins off before checking the next value
    for pin in range (18,22):
     GPIO.output(pin,GPIO.LOW)
    
    #GPIO.output(pin,GPIO.LOW)
    time.sleep(1)


# Turn all LEDS (pins) Off

In [9]:
for pin in range (18,22):
     GPIO.output(pin,GPIO.LOW)